In [ ]:
import os, sys
project_dir = os.path.join(os.getcwd(),'..')
if project_dir not in sys.path:
    sys.path.append(project_dir)

attention_dir = os.path.join(project_dir, 'modules/AttentionMap')
if attention_dir not in sys.path:
    sys.path.append(attention_dir)

sparse_dir = os.path.join(project_dir, 'modules/Sparse')
if sparse_dir not in sys.path:
    sys.path.append(sparse_dir) 

import numpy as np
import os

In [ ]:
import config
root_dir = config.DATASET_DIR
labels = [4]
oimpath = []
y = []
impath = []
for label in np.unique(labels):
    for name in os.listdir(os.path.join(root_dir,'4000-6000_raw',str(label))):
        oimpath.append(os.path.join(root_dir,'4000-6000_raw',str(label),name))
        impath.append(os.path.join(root_dir,str(label),name))
    y = oimpath + [label]*len(os.listdir(os.path.join(root_dir,'4000-6000_raw',str(label))))

In [ ]:
print(len(oimpath),len(impath),oimpath[0],impath[0])

In [ ]:
import cv2

# https://www.kaggle.com/antocad/siim-isic-image-preprocessing-dull-razor
def dullrazor(img, lowbound=15, filterstruc=3, inpaintmat=3, showimgs=True):
    #grayscale
    imgtmp1 = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    #applying a blackhat
    filterSize = (filterstruc, filterstruc)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, filterSize) 
    imgtmp2 = cv2.morphologyEx(imgtmp1, cv2.MORPH_BLACKHAT, kernel)
    #0=skin and 255=hair
    ret, mask = cv2.threshold(imgtmp2, lowbound, 255, cv2.THRESH_BINARY)
    #inpainting
    img_final = cv2.inpaint(img, mask, inpaintmat ,cv2.INPAINT_TELEA)

    if showimgs:
        print("_____DULLRAZOR_____")
        plt.imshow(img)
        plt.show()
        plt.imshow(img_final)
        plt.show()
        plt.imshow(imgtmp1, cmap="gray")
        plt.show()
        plt.imshow(imgtmp2, cmap='gray')
        plt.show()
        plt.imshow(mask, cmap='gray')
        plt.show()
    return img_final

In [ ]:
# PRUEBAS PREVIAS

import matplotlib.pyplot as plt
from torchvision.transforms.functional import resize

from PIL import Image

#idx = 31252+17
idx = 16+1+1
#image_with_hair = Image.open(oimpath[idx])
image_with_hair = Image.open(oimpath[idx]).convert('RGB')

#size_0 = [512,768]
size_0 = [1000,1500]
# size_0 = [2000,3000] # No viable. Olvidate.
image_with_hair = resize(image_with_hair,size_0)

image_nohair = dullrazor(np.array(image_with_hair),lowbound=20, filterstruc=16, inpaintmat=7, showimgs=True)
final_image = Image.fromarray(image_nohair)

#size = [128,128]
#final_image = resize(Image.fromarray(image_nohair),size)

final_image.save(impath[idx])

#plt.imshow(final_image)
#plt.show()

In [ ]:
# GUARDAR IMAGENES CON LOS PARAMS ESTIMADOS ANTES

from PIL import Image
from torchvision.transforms.functional import resize

size = [128,128]

#size_0 = [512,768]
size_0 = [1000,1500]

if size_0 == [512,768]:
    lowbound = 20
    filterstruc = 6
    inpaintmat = 7
elif size_0 == [1000,1500]:
    lowbound = 20
    filterstruc = 16
    inpaintmat = 5


for i in range(len(oimpath)):
    image = Image.open(oimpath[i]).convert('RGB')
    image = resize(image,size_0)
    image = dullrazor(np.array(image),lowbound=lowbound, filterstruc=filterstruc, inpaintmat=inpaintmat, showimgs=False)
    new_image = Image.fromarray(image)
#    new_image = resize(image,size)
    new_image.save(impath[i])